In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
import joblib


In [3]:
df = pd.read_csv("/kaggle/input/credit-scores/credit_scores.csv")

In [4]:
print(df.columns)

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score', 'Count_Auto Loan', 'Count_Credit-Builder Loan',
       'Count_Personal Loan', 'Count_Home Equity Loan', 'Count_Not Specified',
       'Count_Mortgage Loan', 'Count_Student Loan',
       'Count_Debt Consolidation Loan', 'Count_Payday Loan'],
      dtype='object')


In [5]:
df = df.drop(["Name" ,"SSN", "ID", "Customer_ID"], axis=1)

In [6]:
target = "Credit_Score"
x = df.drop(columns=[target])
y = df[target]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=1)

In [8]:
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])

In [9]:
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [10]:
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, selector(dtype_exclude="object")),
    ('cat', categorical_pipeline, selector(dtype_include="object"))
])

In [11]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_classif, k=10))
])

In [12]:
pipeline.fit(x_train, y_train)
x_train_processed = pipeline.transform(x_train)
x_test_processed = pipeline.transform(x_test)

In [13]:
#pipeline.fit(x_train, y_train)

In [14]:
svm = SVC(kernel='rbf')

In [15]:
param_distributions = {
    'C': [ 0.01, 10, 20],
    'gamma': ['scale', 'auto']
             }
    

In [16]:
x_train_sub, _, y_train_sub, _ = train_test_split(x_train_processed, y_train, train_size=0.25, random_state=1)

In [17]:
random_search = RandomizedSearchCV(estimator=svm, param_distributions=param_distributions, n_iter=6, cv=3, scoring='accuracy', n_jobs=-1, random_state=1)

In [18]:
random_search.fit(x_train_processed, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=6, n_jobs=-1,
                   param_distributions={'C': [0.01, 10, 20],
                                        'gamma': ['scale', 'auto']},
                   random_state=1, scoring='accuracy')

In [19]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(x_test_processed)
accuracy = accuracy_score(y_test, y_pred)

In [20]:
print(f"Best Parameters: {random_search.best_params_}")
print(f"Test Accuracy: {accuracy:.4f}")

Best Parameters: {'gamma': 'scale', 'C': 20}
Test Accuracy: 0.6848


In [21]:
pipeline.fit(x, y)
x_processed = pipeline.transform(x)
best_model.fit(x_processed, y)

SVC(C=20)

In [24]:
joblib.dump(pipeline, 'pipeline.pkl')
joblib.dump(best_model, 'svm_model.pkl')

['svm_model.pkl']